StepFun 图片理解最佳实践（Jupyter 笔记）

本笔记基于 StepFun 官方文档“图片理解最佳实践”，整理了常用用法与示例代码，方便快速上手与复用。

参考文档：
- [图片理解最佳实践](https://platform.stepfun.com/docs/guide/image_chat)

内容涵盖：
- 简单图片理解（传入单张图片）
- 多轮对话与多图描述策略
- `detail` 参数的使用（low/high）
- 图片优化（resize、压缩）以降低首字延时
- 透明 PNG 适配（RGBA 转 RGB 白底）
- 以 Base64 方式传图
- 常见问题与限制说明

> 目前推荐使用 step-1o-turbo-vision 模型。该模型拥有最强的视频理解能力，推荐开启high detail 模式。


# 一、实现图片理解
初始化配置

In [ ]:
# 环境依赖与客户端初始化
# - 需要 openai>=1.30.0 （API 兼容 StepFun）
# - 需要设置环境变量 API_KEY

import os
from openai import OpenAI

STEPFUN_API_KEY = os.environ.get("STEPFUN_API_KEY")
if not STEPFUN_API_KEY:
    raise RuntimeError("未检测到环境变量 API_KEY，请在运行前设置，例如：export API_KEY='sk-xxx'")

# 指向 StepFun 平台
client = OpenAI(api_key=STEPFUN_API_KEY, base_url="https://api.stepfun.com/v1")

DEFAULT_VISION_MODEL = "step-1o-turbo-vision"
# DEFAULT_VISION_MODEL="step-1o-vision-32k"
system_prompt = "你是由阶跃星辰提供的AI聊天助手，你除了擅长中文，英文，以及多种其他语言的对话以外，还能够根据用户提供的图片，对内容进行精准的内容文本描述。在保证用户数据安全的前提下，你能对用户的问题和请求，作出快速和精准的回答。同时，你的回答和建议应该拒绝黄赌毒，暴力恐怖主义的内容"




###  1.1简单图片理解

In [ ]:
# 简单图片理解示例
# 传入一张图片并请求描述
# 阶跃星辰支持在 image_url 类型中使用 URL 或 Base64 格式的内容，为了保证更好的性能，推荐使用 URL 来完成图片参数的传递

TEST_IMAGE_URL = "https://q0.itc.cn/q_70/images01/20240612/8e0b7aecb4984be19faaa78f4ecd7c92.jpeg"
completion = client.chat.completions.create(
    model=DEFAULT_VISION_MODEL,
    messages=[
        {"role": "system", "content": system_prompt},
        {
            "role": "user",
            "content": [
                {"type": "text", "text": "用优雅的语言描述这张图片"},
                {
                    "type": "image_url",
                    "image_url": {"url": TEST_IMAGE_URL},
                },
            ],
        },
    ],
)
print("traceid:",completion.id.split('.', 1)[0])
print(completion.choices[0].message.content)
#用优雅的语言描述这张图片

a3b99e28da37c12a9eac66e6011f6c04
这张图片中，女士身着一袭黑色抹胸礼服，礼服设计简约而优雅，突显出端庄的气质。她的长发自然垂落，微微卷曲，发丝柔顺地披在肩头，散发出一种温婉的美感。她佩戴着精致的珠宝，耳畔是华丽的长款耳环，闪烁着璀璨的光芒，与颈间的钻石项链相得益彰，更添几分高贵与典雅。她的姿态从容，手指上也点缀着一枚戒指，整体造型完美融合了优雅与时尚。背景色调温暖，衬托出她的气质如兰，宛如一幅动人的画卷。


### 1.2 基于图片的多轮对话

In [26]:
TEST_IMAGE_URL = "https://q0.itc.cn/q_70/images01/20240612/8e0b7aecb4984be19faaa78f4ecd7c92.jpeg"
completion = client.chat.completions.create(
    model=DEFAULT_VISION_MODEL,
    messages=[
        {"role": "system", "content": system_prompt},
        {
            "role": "user",
            "content": [
                {"type": "text", "text": "用优雅的语言描述这张图片"},
                {
                    "type": "image_url",
                    "image_url": {"url": TEST_IMAGE_URL},
                },
            ],
        },
        # 将模型给予的返回重新插入会对话上下文，进行一轮新的对话
        {
          "role":"assistant",
          "content": "这张图片中，女士身着一袭黑色抹胸礼服，礼服设计简约而优雅，突显出端庄的气质。她的长发自然垂落，微微卷曲，发丝柔顺地披在肩头，散发出一种温婉的美感。她佩戴着精致的珠宝，耳畔是华丽的长款耳环，闪烁着璀璨的光芒，与颈间的钻石项链相得益彰，更添几分高贵与典雅。她的姿态从容，手指上也点缀着一枚戒指，整体造型完美融合了优雅与时尚。背景色调温暖，衬托出她的气质如兰，宛如一幅动人的画卷。"
        },
        # 将用户的新问题继续追问大模型
        {
          "role":"user",
          "content":"那这张照片中的女性大概多少岁？"
        }
    ],
)
# print("traceid:",completion.id.split('.', 1)[0])
# print(completion.choices[0].message.content)
print(completion.model_dump_json(indent=3))


{
   "id": "99275e723d50d11fda8d611d79f53e00.c3af882f7e952496c09645a464591244",
   "choices": [
      {
         "finish_reason": "stop",
         "index": 0,
         "logprobs": null,
         "message": {
            "content": "从图片来看，这位女性的外貌显得非常年轻和精致，皮肤光滑，面部线条柔和，整体状态保养得很好，看起来大约在 **25岁到35岁** 之间。然而，这只是一个基于外貌的推测，实际年龄可能会有所不同。她的妆容和打扮也让她看起来更加优雅和成熟，可能比实际年龄显得更有气质。",
            "refusal": null,
            "role": "assistant",
            "annotations": null,
            "audio": null,
            "function_call": null,
            "tool_calls": null
         }
      }
   ],
   "created": 1757904719,
   "model": "step-1o-turbo-vision",
   "object": "chat.completion",
   "service_tier": null,
   "system_fingerprint": null,
   "usage": {
      "completion_tokens": 79,
      "prompt_tokens": 420,
      "total_tokens": 499,
      "completion_tokens_details": null,
      "prompt_tokens_details": null
   }
}


### 1.3 设置detail参数为low和high

In [ ]:

# step-1v 默认会选择低分辨率，每张图片 400 token
# step-1o 系列模型低分辨率情况下，默认每张图片 169 token；当 detail 模式为 high 时，图片的 Token 消耗将会基于图片大小进行计算

#三国
TEST_IMAGE_URL = "https://camo.githubusercontent.com/9256d51042b0a437e5ab8ec544913cf60aa96e20aca96625491b616cbf53be29/68747470733a2f2f75706c6f61642d696d616765732e6a69616e7368752e696f2f75706c6f61645f696d616765732f31333731343434382d653661316334633862666266303437312e706e673f696d6167654d6f6772322f6175746f2d6f7269656e742f7374726970253743696d61676556696577322f322f772f31323430"

def describe_image(img_url: str, detail: str = "high") -> str:
    messages = [
        {"role": "system", "content": system_prompt},
        {
            "role": "user",
            "content": [
                {"type": "text", "text": "识别图中的词（不要幻想）计数"},
                {"type": "image_url", "image_url": {"url": img_url, "detail": detail}},
            ],
        },
    ]
    completion = client.chat.completions.create(model=DEFAULT_VISION_MODEL, messages=messages,temperature= 0)
    print(completion)
    return completion.choices[0].message.content

# 示例：对单张图先生成描述，然后把描述接入真正对话
context_messages = []
image_desc_low = describe_image(TEST_IMAGE_URL, detail="low")
print("\nlow detail模式:",image_desc_low,"...\n")
image_desc_high = describe_image(TEST_IMAGE_URL, detail="high")
print("\nhigh detail模式:",image_desc_high, "...\n")
# context_messages.append({"role": "user", "content": image_desc})



ChatCompletion(id='5afae40f5e803d636d81e51bba505914.27787464cedad71f620f6ae93b7da118', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='图中的词及其计数如下（按出现顺序）：\n\n**主公**  \n**不能**  \n**幕后**  \n**后主**  \n**汉中**  \n**徐州**  \n**何人**  \n**却说**  \n**丞相**  \n**军士**  \n**如此**  \n**天子**  \n**曹洪**  \n**曹休**  \n**曹真**  \n**众人**  \n**只见**  \n**百姓**  \n**张飞**  \n**关某**  \n**关某在此**  \n**军中**  \n**不可**  \n**左右**  \n**接应**  \n**荆州**  \n**云长**  \n**吕布**  \n**商议**  \n**鲁肃**  \n**周瑜**  \n**相见**  \n**司马懿**  \n**姜维**  \n**军师**  \n**先锋**  \n**董卓**  \n**夏侯惇**  \n**夏侯渊**  \n**曹操**  \n**孔明**  \n**赵云**  \n**明日**  \n**刘备**  \n**大喜**  \n**起兵**  \n**前来**  \n**亲自**  \n**三十万**  \n**先锋**  \n**魏兵**  \n**魏延**  \n**于是东吴**  \n**孔明曰**  \n**何故**  \n**太守**  \n**军令状**  \n**孙权**  \n**孙刘之兵**  \n**太师**  \n**大惊**  \n**众人将令**  \n**不知**  \n**此乃**  \n**此战**  \n**先生**  \n**先生高见**  \n**马超**  \n**出马**  \n**星夜**  \n**一面**  \n**一面**  \n**然后**  \n**兵将**  \n**夏侯渊**  \n**兴兵**  \n**大军**  \n**必

### 1.4 多图图片理解

根据模型不同，一次多轮对话最多可以拥有不超过 10 张照片或 50 张照片
当图片数量可能超过上限（当前单次请求上限为 60 张）时，可先用模型为每张图生成描述，然后将描述作为对话上下文，继续进行真正的问答。

In [34]:


completion = client.chat.completions.create(
  model="step-1o-turbo-vision",
  messages=[
        {"role": "system", "content": system_prompt},

      # 在对话中传入图片，来实现基于图片的理解
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": "用优雅的语言描述这几张照片",
                },
                {
                    "type": "image_url",
                    "image_url": {
                        "url": "https://q0.itc.cn/q_70/images01/20240612/8e0b7aecb4984be19faaa78f4ecd7c92.jpeg"
                    },
                },
                {
                    "type":"image_url",
                    "image_url":{
                        "url":"https://gd-hbimg.huaban.com/2b4d0f9fc1515d4d1d2d498b8318dbac3491cc58175ee-83PEcb_fw1200webp"
                    }
                },
                {
                    "type":"image_url",
                    "image_url":{
                        "url":"https://gd-hbimg.huaban.com/30576c5c513d28154df3cee1570027bb95a47bc0413d4-6sNdZc_fw1200webp"
                    }
                },
                {
                    "type":"image_url",
                    "image_url":{
                        "url":"https://gd-hbimg.huaban.com/14eb4eb2e87b9a46973e9ddca872750043375ad02818c-ildFmT_fw1200webp"
                    }
                }
            ],
      },
  ],
)
 
print(completion.model_dump_json(indent=3))


{
   "id": "7f059cdd01dd3b9d91be29ec18edaae8.298794de572a5a449040b508cf24f06b",
   "choices": [
      {
         "finish_reason": "stop",
         "index": 0,
         "logprobs": null,
         "message": {
            "content": "这几张照片展现了女性的优雅与魅力，每一张都散发着独特的气质与风格。  \n\n### 第一张照片  \n她身着一袭黑色礼服，简约而大方，抹胸的设计突显出端庄与从容。黑色的深邃与她的长发相得益彰，发丝柔顺地垂落，带着几分温婉。她佩戴的珠宝——耳环与项链，设计精致，闪烁着低调的光芒，为整体造型增添了一丝奢华感。她的姿态沉稳，目光坚定，仿佛在静静诉说着内心的力量与自信。  \n\n### 第二张照片  \n她换上了一袭清新的浅绿色礼服，礼服上点缀着精致的亮片，折射出柔和的光芒，宛如晨曦中的露珠。她的发型优雅地盘起，几缕碎发轻柔地垂在耳边，增添了几分柔美。她手中拿着一张写有“哈根达斯『幂』月之旅机票”的卡片，脸上带着淡淡的微笑，眼神中透着温柔与亲切，仿佛在邀请人们进入一个梦幻的世界。  \n\n### 第三张照片  \n这张照片充满了力量与个性。她身着一件缀满亮片的银色抹胸礼服，礼服在光线下闪耀着璀璨的光芒，宛如星空般耀眼。外搭的黑白毛绒外套为整体造型增添了一丝野性与不羁，与她红唇的妆容相辅相成，展现出强烈的视觉冲击力。她的姿态自信而张扬，高高束起的马尾更凸显出她的干练与果敢，仿佛是一位掌控全局的女王。  \n\n### 第四张照片  \n她身穿一袭深蓝色丝绒礼服，礼服的质感光滑而高级，吊带的设计突显出她的锁骨与肩部线条，优雅中带着一丝性感。外层的浅蓝色薄纱如流水般垂落，为整体造型增添了一丝仙气与梦幻感。她站在绿意盎然的背景前，黑发如瀑，随风轻扬，双手在胸前交握，姿态端庄而温柔。她的眼神平静而深邃，仿佛与自然融为一体，散发出一种宁静而高雅的气质。  \n\n### 总体感受  \n这组照片通过不同的服装、妆容与背景，展现了女性的多面魅力——时而端庄大方，时而温柔亲切，时而个性张扬，时而宁静高雅。每一张照片都像是一幅精心

### 1.5 使用 Base64 编码来传递图片内容
![image.png](https://platform.stepfun.com/_next/image?url=%2F_next%2Fstatic%2Fmedia%2Fbase64image.f0a0bc95.jpg&w=3840&q=75)

In [ ]:
# 以 Base64 输入图片
import base64
# 01_鱼香肉丝.jpg 识别图中的菜 jpeg
with open("./media/test-监测.png", "rb") as image_file:
    base64_bytes = base64.b64encode(image_file.read())
    base64_bytes = base64_bytes.decode('utf-8')

completion_b64 = client.chat.completions.create(
    model=DEFAULT_VISION_MODEL,
    messages=[
        {"role": "system", "content": system_prompt},
        {
            "role": "user",
            "content": [
                {"type": "text", "text": "分析图片，识别每一项的结果和参考范围，如参考范围为3.00-5.00，2.50(低于3.00)和6.50(高于5.00的检验结果为异常"},
                {
                    "type": "image_url",
                    "image_url": {
                        "url": f"data:image/png;base64,{base64_bytes}",
                        "detail": "high",
                    },

                },
            ],
        },
    ],
)

print(completion_b64)
# print(completion_b64.choices[0].message.content)


## 二、常见问题处理

### 2.1 图片预处理以降低延迟
预处理：对图片缩放与压缩来获得较好的处理速度
- step-1o 对于 detail=low 或默认，可将最长边缩放至 728px
- step-1o 对于 detail=high，可将最长边缩放至 504 的倍数（示例用 2688）
- step-1v 对于 detail=low 或默认，可将最长边缩放至 1280px
- step-1v 对于 detail=high，可将最长边缩放至 2688px

In [ ]:

from PIL import Image
# 确保安装了Pillow库
# 运行以下命令以安装Pillow
# pip install Pillow
 
 
# 处理图片为 80% 质量
 
def compress(input_path, output_path, quality):
    image = Image.open(input_path)
    image.save(output_path, quality=quality,optimize=True)
 
# 将图片按照最长边resize到2688，短边等比例缩放，并存储为新图片
def resize_image(input_path, output_path, max_size):
    image = Image.open(input_path)
    width, height = image.size
 
    # 计算新的尺寸
    if width > height:
        new_width = max_size
        new_height = int((max_size / width) * height)
    else:
        new_height = max_size
        new_width = int((max_size / height) * width)
 
    # 调整图片尺寸
    resized_image = image.resize((new_width, new_height), Image.Resampling.LANCZOS)
    resized_image.save(output_path)
 
# 调用函数
# resize_image('./media/03_视力表.jpg', './media/03_视力表_resized.jpg', 2688)
compress('./media/03_视力表.jpg', './media/03_视力表_re_quality.jpg', 80)


/opt/miniconda3/envs/py311/lib/python3.11/site-packages/PIL/Image.py:3452: DecompressionBombWarning: Image size (93752978 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


### 2.2 透明背景图片
目前 step-1o 系列的模型支持对透明背景的 PNG 图片进行处理，但在使用时，会将透明通道处理景为黑色的情况。你可以通过对图片进行处理，将其背景设置为白色，从而避免模型在推理时，无法正确理解图片中的内容。

In [45]:
# 透明 PNG 适配：RGBA 转 RGB 白色背景
# https://gd-hbimg.huaban.com/7c4795bc026c359c8d17ff19d1d25b574fbb43e03088e9-ena8d7_fw1200webp
from PIL import Image

def convert_rgba_to_rgb_with_white_background(input_path: str, output_path: str) -> None:
    img = Image.open(input_path)
    if img.mode != "RGBA":
        raise ValueError("输入图片不是 RGBA 模式")
    white_background = Image.new("RGB", img.size, (255, 255, 255))
    white_background.paste(img, mask=img.split()[3])
    result = white_background.convert("RGB")
    result.save(output_path)

convert_rgba_to_rgb_with_white_background("./media/03_rgba.webp", "./media/03_rgb.webp")


## 三、其他应用

### 3.1 识别图中的单个主体并图片标注

In [ ]:
import os
import base64
import json
import time
from typing import Dict, Any, Tuple, Optional, List

from PIL import Image, ImageDraw, ImageFont
from openai import OpenAI

STEP_API_KEY = os.environ["STEPFUN_API_KEY"]
BASE_URL= os.environ['STEPFUN_ENDPOINT']


MODEL_NAME = "step-1o-vision-32k"
RESPONSE_FORMAT_JSON = {"type": "json_object"}

# 坐标模式（用于可视化落地到像素）：强制相对坐标（0~1000）
POSITION_MODE = "relative_0_1000"

# 输入/输出
INPUT_IMAGE_PATH = "./media/03_object.webp"
OUTPUT_IMAGE_DIR = "./media"     # 目录；程序会自动生成文件名
DRAW_LABEL = "crop"                         # 在图上绘制的框标签
FONT_PATHS = [
    "/System/Library/Fonts/Supplemental/Arial Unicode.ttf",
    "/System/Library/Fonts/Helvetica.ttc",
    "arial.ttf",
]
FONT_SIZE = 20

# 用户自然语言主体描述（示例：请按需修改）
SUBJECT_DESCRIPTION = "图中主要的菜品主体，完整包含并保留少量上下文边缘空间"

# 重试
MAX_RETRIES = 5
RETRY_SLEEP_SECONDS = 1

# 调试输出
VERBOSE = True
# =========================
# ===== 配置区域结束 ======
# =========================


client = OpenAI(api_key=STEP_API_KEY, base_url=BASE_URL)


def log(*args):
    if VERBOSE:
        print(*args)


# =========================
# ===== 提示词（Prompt） ===
# =========================
def build_system_prompt() -> str:
    """
    严格约束输出为单一裁剪框的 JSON：
    {
      "top_left": [x1, y1],
      "bottom_right": [x2, y2],
      "confidence": 0.95
    }
    """
    return (
        "你是一个专业的图像分析助手。你的任务是：根据用户的自然语言描述，"
        "在给定图片中定位需要裁剪的**核心主体**区域，并仅输出一个裁剪框。\n\n"
        "【输出格式（必须严格遵守）】\n"
        "只输出一个 JSON 对象，不要包含任何其他解释或文字：\n"
        "{\n"
        "  \"top_left\": [x1, y1],\n"
        "  \"bottom_right\": [x2, y2],\n"
        "  \"confidence\": 0.95\n"
        "}\n\n"
        "【坐标与边界规则】\n"
        "1) 使用**相对坐标制**：范围 0~1000 的整数；(0,0) 为左上角，x 向右、y 向下。\n"
        "2) 所有坐标必须位于图像范围内，且满足 x1 < x2、y1 < y2。\n"
        "3) 你的输出只有**一个**裁剪框；如存在多个可能主体，应将它们**合并为一个更大框**以避免重叠问题。\n"
        "4) 框需要**完整包含**用户描述的主体，同时**适度保留上下文**，避免过紧贴边缘。\n"
        "5) 置信度 confidence 取值 0.0–1.0，代表你对本次定位准确度的判断。\n\n"
        "【一致性要求】\n"
        "- 坐标字段名固定为 \"top_left\" 与 \"bottom_right\"；值均为两个整数 [x, y]，范围 0–1000。\n"
        "- 输出中不得包含额外字段、注释或解释。\n"
    )


# =========================
# ====== 工具函数 =========
# =========================
def get_image_size(image_path: str) -> Tuple[int, int]:
    with Image.open(image_path) as img:
        return img.size  # (w, h)


def image_to_base64(image_path: str) -> str:
    with open(image_path, "rb") as f:
        return base64.b64encode(f.read()).decode("utf-8")


def _clamp(v: int, low: int, high: int) -> int:
    return max(low, min(v, high))


def _to_int2(pt: List[Any]) -> Optional[Tuple[int, int]]:
    if not isinstance(pt, (list, tuple)) or len(pt) != 2:
        return None
    try:
        x = int(round(float(pt[0])))
        y = int(round(float(pt[1])))
        return x, y
    except Exception:
        return None


def validate_relative_bbox(bbox: Dict[str, Any]) -> bool:
    """
    校验相对坐标 0..1000，x1<x2, y1<y2
    """
    tl = _to_int2(bbox.get("top_left"))
    br = _to_int2(bbox.get("bottom_right"))
    if tl is None or br is None:
        return False
    x1, y1 = tl
    x2, y2 = br
    if not (0 <= x1 < x2 <= 1000 and 0 <= y1 < y2 <= 1000):
        return False
    conf = bbox.get("confidence")
    try:
        conf = float(conf)
    except Exception:
        return False
    return 0.0 <= conf <= 1.0


def rel_to_abs_bbox(bbox: Dict[str, Any], img_size: Tuple[int, int]) -> Tuple[int, int, int, int]:
    """
    将相对 0..1000 的坐标转换为像素坐标（用于可视化）
    """
    w, h = img_size
    x1_rel, y1_rel = bbox["top_left"]
    x2_rel, y2_rel = bbox["bottom_right"]
    x1 = int(round(x1_rel / 1000.0 * w))
    y1 = int(round(y1_rel / 1000.0 * h))
    x2 = int(round(x2_rel / 1000.0 * w))
    y2 = int(round(y2_rel / 1000.0 * h))
    # 夹取并保证有效
    x1 = _clamp(x1, 0, w - 2)
    y1 = _clamp(y1, 0, h - 2)
    x2 = _clamp(max(x2, x1 + 1), 1, w - 1)
    y2 = _clamp(max(y2, y1 + 1), 1, h - 1)
    return x1, y1, x2, y2


def _load_font(paths: List[str], size: int) -> ImageFont.FreeTypeFont:
    for p in paths:
        try:
            return ImageFont.truetype(p, size=size)
        except Exception:
            continue
    return ImageFont.load_default()


def annotate_image(image_path: str, bbox_abs: Tuple[int, int, int, int], output_path: str, label: str = "crop") -> None:
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    with Image.open(image_path) as img:
        if img.mode != "RGBA":
            img = img.convert("RGBA")

        draw = ImageDraw.Draw(img)
        font = _load_font(FONT_PATHS, FONT_SIZE)

        x1, y1, x2, y2 = bbox_abs
        draw.rectangle([x1, y1, x2, y2], outline=(255, 0, 0, 255), width=2)

        # 居中标签
        cx, cy = (x1 + x2) // 2, (y1 + y2) // 2
        l, t, r, b = draw.textbbox((0, 0), label, font=font)
        tw, th = r - l, b - t
        pad = 6
        rect = [cx - tw // 2 - pad, cy - th // 2 - pad, cx + tw // 2 + pad, cy + th // 2 + pad]
        W, H = img.size
        # 夹取
        dx = -min(0, rect[0]) + min(0, W - rect[2])
        dy = -min(0, rect[1]) + min(0, H - rect[3])
        rect = [rect[0] + dx, rect[1] + dy, rect[2] + dx, rect[3] + dy]

        draw.rectangle(rect, fill=(0, 0, 0, 140))
        draw.text((rect[0] + pad, rect[1] + pad), label, fill=(255, 255, 255, 255), font=font)

        ext = os.path.splitext(output_path)[1].lower()
        if ext in [".jpg", ".jpeg"]:
            background = Image.new("RGB", img.size, (255, 255, 255))
            background.paste(img, mask=img.split()[3])
            background.save(output_path, quality=95)
        else:
            img.save(output_path)

    log(f"标注后的图像已保存至 {output_path}")


# =========================
# ====== 推理请求 =========
# =========================
def call_model(image_b64: str, description: str) -> Optional[Dict[str, Any]]:
    sys_prompt = build_system_prompt()
    log("系统提示内容:\n\n", sys_prompt)

    messages = [
        {"role": "system", "content": sys_prompt},
        {
            "role": "user",
            "content": [
                {"type": "text", "text": f"主体描述：{description}"},
                {
                    "type": "image_url",
                    "image_url": {
                        "url": f"data:image/jpg;base64,{image_b64}",
                        "detail": "high",
                    },
                },
                {"type": "text", "text": "请基于以上描述与图像，返回唯一的裁剪框 JSON。"},
            ],
        },
    ]

    try:
        response = client.chat.completions.create(
            messages=messages,
            model=MODEL_NAME,
            response_format=RESPONSE_FORMAT_JSON,
        )
    except Exception as e:
        log(f"API请求失败: {e}")
        return None

    # 打印完整响应（可按需注释）
    log(response)

    try:
        content = response.choices[0].message.content.strip()
        log("原始返回：", content)
        data = json.loads(content)
        return data
    except Exception as e:
        log("解析JSON失败，请检查API返回内容。报错：", e)
        return None


# =========================
# ========= 主流程 =========
# =========================
def main():
    img_w, img_h = get_image_size(INPUT_IMAGE_PATH)
    log(f"图像大小: {(img_w, img_h)}")

    image_b64 = image_to_base64(INPUT_IMAGE_PATH)

    attempt = 0
    while attempt < MAX_RETRIES:
        try:
            result = call_model(image_b64, SUBJECT_DESCRIPTION)
            if not result:
                log("未能获取有效的分析结果。")
                attempt += 1
                time.sleep(RETRY_SLEEP_SECONDS)
                continue

            # 校验相对坐标与结构
            if not validate_relative_bbox(result):
                log("返回的坐标不符合 0..1000 或结构不正确，重试...")
                attempt += 1
                time.sleep(RETRY_SLEEP_SECONDS)
                continue

            # 相对 -> 像素（仅用于可视化）
            x1, y1, x2, y2 = rel_to_abs_bbox(result, (img_w, img_h))

            # 生成输出文件名
            os.makedirs(OUTPUT_IMAGE_DIR, exist_ok=True)
            base_name = os.path.splitext(os.path.basename(INPUT_IMAGE_PATH))[0]
            timestamp = time.strftime("%Y%m%d_%H%M%S")
            file_name = f"{base_name}_{MODEL_NAME}_{timestamp}.png"
            output_path = os.path.join(OUTPUT_IMAGE_DIR, file_name)

            annotate_image(INPUT_IMAGE_PATH, (x1, y1, x2, y2), output_path, DRAW_LABEL)

            # 控制台也打印一次标准 JSON，便于直接消费
            # 注意：此处打印的就是“相对坐标 0..1000”的原始返回
            print(json.dumps({
                "top_left": [int(result["top_left"][0]), int(result["top_left"][1])],
                "bottom_right": [int(result["bottom_right"][0]), int(result["bottom_right"][1])],
                "confidence": float(result["confidence"])
            }, ensure_ascii=False))
            return

        except Exception as e:
            log(f"发生错误: {e}")
            attempt += 1
            time.sleep(RETRY_SLEEP_SECONDS)

    log("达到最大重试次数，操作失败。")


if __name__ == "__main__":
    main()

图像大小: (2048, 2048)
系统提示内容:

 你是一个专业的图像分析助手。你的任务是：根据用户的自然语言描述，在给定图片中定位需要裁剪的**核心主体**区域，并仅输出一个裁剪框。

【输出格式（必须严格遵守）】
只输出一个 JSON 对象，不要包含任何其他解释或文字：
{
  "top_left": [x1, y1],
  "bottom_right": [x2, y2],
  "confidence": 0.95
}

【坐标与边界规则】
1) 使用**相对坐标制**：范围 0~1000 的整数；(0,0) 为左上角，x 向右、y 向下。
2) 所有坐标必须位于图像范围内，且满足 x1 < x2、y1 < y2。
3) 你的输出只有**一个**裁剪框；如存在多个可能主体，应将它们**合并为一个更大框**以避免重叠问题。
4) 框需要**完整包含**用户描述的主体，同时**适度保留上下文**，避免过紧贴边缘。
5) 置信度 confidence 取值 0.0–1.0，代表你对本次定位准确度的判断。

【一致性要求】
- 坐标字段名固定为 "top_left" 与 "bottom_right"；值均为两个整数 [x, y]，范围 0–1000。
- 输出中不得包含额外字段、注释或解释。

ChatCompletion(id='6cf1faf6d0934d69b19ae81cbbce7c9b.6cc669845e5de4e3aa732460bcc53982', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='{"top_left": [18, 18], "bottom_right": [982, 982], "confidence": 0.99}', refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=None))], created=1758785879, model='step-1o-vision-32k', object='chat.completion', servi

### 3.2 识别图片中的多个物体并标注

In [43]:
import os
import base64
import json
import time
from typing import Dict, Any, Tuple, Optional, List

from PIL import Image, ImageDraw, ImageFont
from openai import OpenAI

STEP_API_KEY = os.environ["STEPFUN_API_KEY"]
BASE_URL= os.environ['STEPFUN_ENDPOINT']


MODEL_NAME = "step-1o-vision-32k"
RESPONSE_FORMAT_JSON = {"type": "json_object"}

# 坐标模式（用于可视化落地到像素）：强制相对坐标（0~1000）
POSITION_MODE = "relative_0_1000"

# 输入/输出
# INPUT_IMAGE_PATH = "./media/03_2.jpg"
# OUTPUT_IMAGE_PATH = "./media/03_2_output_annotated.png"  

INPUT_IMAGE_PATH = "./media/03_6.png"
OUTPUT_IMAGE_PATH = "./media/03_6_output_annotated.png"  
DRAW_LABEL = "crop"                         # 在图上绘制的框标签
FONT_PATHS = [
    "/System/Library/Fonts/Supplemental/Arial Unicode.ttf",
    "/System/Library/Fonts/Helvetica.ttc",
    "arial.ttf",
]
FONT_SIZE = 20

# 用户自然语言主体描述（示例：请按需修改）
SUBJECT_DESCRIPTION = "识别图中的文本主体" #"图中主要的食材主体，注意要完整包含并保留少量上下文边缘空间"

# 调试输出
VERBOSE = True


client = OpenAI(api_key=STEP_API_KEY, base_url=BASE_URL)


def log(*args):
    if VERBOSE:
        print(*args)


# =========================
# ===== 提示词（Prompt） ===
# =========================
def build_system_prompt() -> str:
 
    return (
        """
        你是专业图像分析助手，需精准分析用户提供的图片，根据其描述识别所有指定主体区域，为每个主体输出准确包含主体的边界框坐标。

        ### 输出格式（严格遵循）：
        仅输出JSON格式的字符串数组，每个元素为"x1,y1,x2,y2"（左上角x1,y1，右下角x2,y2），无任何额外文字。示例：
        ["ax1,ay1,ax2,ay2","bx1,by1,bx2,by2"]

        ### 坐标规则：
        1. 坐标系：以图片左上角为原点(0,0)，x轴向右递增，y轴向下递增
        2. 数值标准：采用相对坐标（0-1000整数），0为最左/最上，1000为最右/最下（需符合图片比例）
        3. 有效性：确保x1 < x2、y1 < y2，且所有坐标在0-1000范围内

        ### 核心要求：
        - **比例匹配**：边界框的宽高比（x2-x1 : y2-y1）必须接近主体本身的自然宽高比
        - **中心一致**：物体的中心和边界框的中心应尽量重合，
        - **边框接近**：框的上、下、左、右边缘均需**紧挨**主体的自然轮廓，刚好把主体框住，与主体边缘贴合没有缝隙

        ### 错误规避：
        - 边缘离主体过远，导致框内包含大量空白
        - 边缘切割主体，导致主体部分缺失
        """
 )

        # ### 核心要求：
        # - **比例匹配**：边界框的宽高比（x2-x1 : y2-y1）必须**接近主体本身的自然宽高比**（如主体是宽扁形，框应横向更宽；主体是瘦高形，框可纵向稍长，但禁止过度拉伸）
        # - **横向足够覆盖**：确保横向（x轴）完整包含主体左右边缘，左右预留空间（x2-x1的2-3%），避免横向过窄导致主体两侧被裁切或挤压
        # - **纵向严格收缩**：纵向（y轴）仅保留主体上下必要空间（y2-y1的1-2%），禁止纵向过度扩展（如主体高度占框高的80%以下视为过宽，需压缩）
        # - **贴合主体轮廓**：框的边缘需对齐主体的自然轮廓（如人物的肩膀宽度、物体的左右边缘），横向以主体最宽处为界，纵向以主体最高/最低点为界

        # ### 错误规避：
        # - 禁止“横向紧卡主体、纵向包含大量空白”的窄长框
        # - 禁止“纵向紧贴主体、横向未覆盖完整”的扁宽框
        # - 输出前必须检查每个框的宽高比是否与主体视觉比例一致
        # 


        # """
        # 角色与任务定义：
        # 你是一个专业的图像分析助手。你的任务是精确分析用户提供的图片，并根据用户的自然语言描述，识别出图片中所有指定的主体区域，并为每个主体输出一个边界框坐标。

        # 输出指令：
        # 请输出一个字符串数组，每个字符串代表一个边界框的坐标，格式为"x1,y1,x2,y2"，其中x1,y1是左上角坐标，x2,y2是右下角坐标。只输出数组内容，不做任何其他解释。
        # 请严格按照以下JSON格式输出裁剪区域的坐标，并且只输出JSON内容，不做任何其他解释。
        # ["ax1,ay1,ax2,ay2","bx1,by1,bx2,by2"]

        # 关键要求：
        # 坐标系统：以图片的左上角为原点 (0, 0)。x轴向右为正，y轴向下为正。
        # 单位：坐标值应为整数像素值，但使用相对坐标系统，范围从0到1000，其中0表示最左/最上，1000表示最右/最下。
        # 完整性：确保每个边界框能够*完整*包含描述的主体，同时可保留合理的周边上下文空间，避免过于紧贴主体。
        # 对于多个主体：为每个检测到的主体输出一个框。如果用户描述多个主体，确保框出所有指定主体。

        # 边界规则：
        # 1) 坐标采用相对坐标制：范围为 0~1000 的整数。
        # 2) 所有坐标必须在图像范围内，且对于每个框，满足 x1 < x2 和 y1 < y2。
        # 3) 不同实物的框之间不得重叠。如可能重叠：
        #     ◦ 优先保留面积更大的、与实物边界更贴合的框；
        #     ◦ 或者将可能重叠的多个框合并为一个更大框（以避免重叠），但仅当它们属于同一类别或用户描述允许时。
        # """
   

# =========================
# ====== 工具函数 =========
# =========================
def get_image_size(image_path: str) -> Tuple[int, int]:
    with Image.open(image_path) as img:
        return img.size  # (w, h)


def image_to_base64(image_path: str) -> str:
    with open(image_path, "rb") as f:
        return base64.b64encode(f.read()).decode("utf-8")


def _clamp(v: int, low: int, high: int) -> int:
    return max(low, min(v, high))


def _to_int2(pt: List[Any]) -> Optional[Tuple[int, int]]:
    if not isinstance(pt, (list, tuple)) or len(pt) != 2:
        return None
    try:
        x = int(round(float(pt[0])))
        y = int(round(float(pt[1])))
        return x, y
    except Exception:
        return None


def validate_relative_bbox(bbox: Dict[str, Any]) -> bool:
    """
    校验相对坐标 0..1000，x1<x2, y1<y2
    """
    tl = _to_int2(bbox.get("top_left"))
    br = _to_int2(bbox.get("bottom_right"))
    if tl is None or br is None:
        return False
    x1, y1 = tl
    x2, y2 = br
    if not (0 <= x1 < x2 <= 1000 and 0 <= y1 < y2 <= 1000):
        return False
    conf = bbox.get("confidence")
    try:
        conf = float(conf)
    except Exception:
        return False
    return 0.0 <= conf <= 1.0


def rel_to_abs_bbox(bbox: Dict[str, Any], img_size: Tuple[int, int]) -> Tuple[int, int, int, int]:
    """
    将相对 0..1000 的坐标转换为像素坐标（用于可视化）
    """
    w, h = img_size
    x1_rel, y1_rel = bbox["top_left"]
    x2_rel, y2_rel = bbox["bottom_right"]
    x1 = int(round(x1_rel / 1000.0 * w))
    y1 = int(round(y1_rel / 1000.0 * h))
    x2 = int(round(x2_rel / 1000.0 * w))
    y2 = int(round(y2_rel / 1000.0 * h))
    # 夹取并保证有效
    x1 = _clamp(x1, 0, w - 2)
    y1 = _clamp(y1, 0, h - 2)
    x2 = _clamp(max(x2, x1 + 1), 1, w - 1)
    y2 = _clamp(max(y2, y1 + 1), 1, h - 1)
    return x1, y1, x2, y2


def _load_font(paths: List[str], size: int) -> ImageFont.FreeTypeFont:
    for p in paths:
        try:
            return ImageFont.truetype(p, size=size)
        except Exception:
            continue
    return ImageFont.load_default()


class ImageAnnotator:
    def __init__(self, image_path: str):
        """初始化标注器，加载图片并准备绘制对象"""
        # 只加载一次图片
        self.image_path = image_path
        self.img = Image.open(image_path)
        # 转换为RGBA以支持透明图层（只做一次）
        if self.img.mode != "RGBA":
            self.img = self.img.convert("RGBA")
        # 创建绘制对象（只初始化一次）
        self.draw = ImageDraw.Draw(self.img)
        # 加载字体（只加载一次）
        self.font = _load_font(FONT_PATHS, FONT_SIZE)

    def add_annotation(self, bbox_abs: Tuple[int, int, int, int], label: str = "crop"):
        """添加一个标注（可多次调用）"""
        x1, y1, x2, y2 = bbox_abs
        
        # 绘制边界框
        self.draw.rectangle([x1, y1, x2, y2], outline=(255, 0, 0, 255), width=2)
        
        # 绘制居中标签
        cx, cy = (x1 + x2) // 2, (y1 + y2) // 2
        l, t, r, b = self.draw.textbbox((0, 0), label, font=self.font)
        tw, th = r - l, b - t
        pad = 6
        rect = [
            cx - tw // 2 - pad, 
            cy - th // 2 - pad, 
            cx + tw // 2 + pad, 
            cy + th // 2 + pad
        ]
        W, H = self.img.size
        
        # 调整标签位置避免超出图像边界
        dx = -min(0, rect[0]) + min(0, W - rect[2])
        dy = -min(0, rect[1]) + min(0, H - rect[3])
        rect = [rect[0] + dx, rect[1] + dy, rect[2] + dx, rect[3] + dy]
        
        # 绘制标签背景和文字
        self.draw.rectangle(rect, fill=(0, 0, 0, 140))
        self.draw.text(
            (rect[0] + pad, rect[1] + pad), 
            label, 
            fill=(255, 255, 255, 255), 
            font=self.font
        )

    def save(self, output_path: str):
        """保存标注后的图片（只保存一次）"""
        os.makedirs(os.path.dirname(output_path), exist_ok=True)
        ext = os.path.splitext(output_path)[1].lower()
        
        # 处理JPG格式的透明背景问题
        if ext in [".jpg", ".jpeg"]:
            background = Image.new("RGB", self.img.size, (255, 255, 255))
            background.paste(self.img, mask=self.img.split()[3])  # 使用alpha通道作为掩码
            background.save(output_path, quality=95)
        else:
            self.img.save(output_path)
        
        # 关闭图片（释放资源）
        self.img.close()
        print(f"标注后的图像已保存至 {output_path}")


def call_model(image_b64: str, description: str) -> Optional[Dict[str, Any]]:
    sys_prompt = build_system_prompt()
    log("系统提示内容:\n\n", sys_prompt)

    messages = [
        {"role": "system", "content": sys_prompt},
        {
            "role": "user",
            "content": [
                {"type": "text", "text": f"主体描述：{description}"},
                {
                    "type": "image_url",
                    "image_url": {
                        "url": f"data:image/jpg;base64,{image_b64}",
                        "detail": "high",
                    },
                },
                {"type": "text", "text": "请基于以上描述与图像，返回所有符合要求的裁剪框数组。"},
            ],
            "temprature": 0.0,
        },
    ]

    try:
        response = client.chat.completions.create(
            messages=messages,
            model=MODEL_NAME,
            response_format=RESPONSE_FORMAT_JSON,
        )
    except Exception as e:
        log(f"API请求失败: {e}")
        return None

    # 打印完整响应（可按需注释）
    log(response)

    try:
        content = response.choices[0].message.content.strip()
        log("原始返回：", content)
        data = json.loads(content)
        return data
    except Exception as e:
        log("解析JSON失败，请检查API返回内容。报错：", e)
        return None


def main():
    img_w, img_h = get_image_size(INPUT_IMAGE_PATH)
    log(f"图像大小: {(img_w, img_h)}")

    image_b64 = image_to_base64(INPUT_IMAGE_PATH)

    result = call_model(image_b64, SUBJECT_DESCRIPTION)
    if not result:
        log("未能获取有效的分析结果。")
    
    annotator = ImageAnnotator(INPUT_IMAGE_PATH)
    print(result)

    for i, item in enumerate(result, 1):
        # 关键修复：字符串按逗号分割，转换为整数元组
        x1_rel, y1_rel, x2_rel, y2_rel = map(float, item.split(','))
    
        # 步骤2：将相对坐标转换为像素坐标（乘以图片宽高）
        x1 = int(x1_rel * img_w/1000.0)  # x方向坐标 × 宽度
        y1 = int(y1_rel * img_h/1000.0)  # y方向坐标 × 高度
        x2 = int(x2_rel * img_w/1000.0)
        y2 = int(y2_rel * img_h/1000.0)
        
        # 转换为元组（像素坐标）
        bbox_abs = (x1, y1, x2, y2)
        # 现在可以正常传递给add_annotation了
        annotator.add_annotation(bbox_abs, label=f"")
        # annotator.add_annotation(bbox_abs, label=f"文本_{i}")
    
    # 3. 最终保存图片（只保存一次）
    annotator.save(OUTPUT_IMAGE_PATH)


if __name__ == "__main__":
    main()

图像大小: (892, 1205)
系统提示内容:

 
        你是专业图像分析助手，需精准分析用户提供的图片，根据其描述识别所有指定主体区域，为每个主体输出准确包含主体的边界框坐标。

        ### 输出格式（严格遵循）：
        仅输出JSON格式的字符串数组，每个元素为"x1,y1,x2,y2"（左上角x1,y1，右下角x2,y2），无任何额外文字。示例：
        ["ax1,ay1,ax2,ay2","bx1,by1,bx2,by2"]

        ### 坐标规则：
        1. 坐标系：以图片左上角为原点(0,0)，x轴向右递增，y轴向下递增
        2. 数值标准：采用相对坐标（0-1000整数），0为最左/最上，1000为最右/最下（需符合图片比例）
        3. 有效性：确保x1 < x2、y1 < y2，且所有坐标在0-1000范围内

        ### 核心要求：
        - **比例匹配**：边界框的宽高比（x2-x1 : y2-y1）必须接近主体本身的自然宽高比
        - **中心一致**：物体的中心和边界框的中心应尽量重合，
        - **边框接近**：框的上、下、左、右边缘均需**紧挨**主体的自然轮廓，刚好把主体框住，与主体边缘贴合没有缝隙

        ### 错误规避：
        - 边缘离主体过远，导致框内包含大量空白
        - 边缘切割主体，导致主体部分缺失
        
ChatCompletion(id='32258c6b8efa1e626fe70c0ed46a9921.5e2478762595e91ed5da73701f8b0a71', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='["150,50,850,950"]', refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=None))], created=175